In [1]:
import os 
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
os.chdir("../")

In [ ]:
df = pd.read_csv("data/telegram_data.csv")

In [ ]:
df

In [ ]:
print("Checking for NaN values in the 'Message' column:")
nan_count = df['Message'].isnull().sum()
print(f"Number of NaN values in 'Message' column: {nan_count}")

In [ ]:
df = df.dropna(subset=['Message'])

# Print the shape of the dataset after dropping NaN values in the "Message" column
print(f"Dataset shape after dropping NaN values in 'Message' column: {df.shape}")

In [ ]:
df.head()

In [ ]:
message_df=df['Message']
message_df

In [ ]:

# Example of your DataFrame
# df = pd.DataFrame({'Message': ['💥3pcs silicon brush spatulas...', '💥Mandoline Slicer...', ...]})

# Define a function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile(
        "[" 
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251" 
        "]+", 
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

# Apply the function to the 'Message' column
df['Message'] = df['Message'].apply(remove_emojis)

# Display the updated DataFrame
print(df.head())


In [ ]:
df.to_csv('clean_data.csv')

In [ ]:
df

In [ ]:
import re

def label_message_utf8_with_birr(message):
    # Split the message at the first occurrence of '\n'
    if '\n' in message:
        first_line, remaining_message = message.split('\n', 1)
    else:
        first_line, remaining_message = message, ""
    
    labeled_tokens = []
    
    # Tokenize the first line
    first_line_tokens = re.findall(r'\S+', first_line)
    
    # Label the first token as B-PRODUCT and the rest as I-PRODUCT
    if first_line_tokens:
        labeled_tokens.append(f"{first_line_tokens[0]} B-PRODUCT")  # First token as B-PRODUCT
        for token in first_line_tokens[1:]:
            labeled_tokens.append(f"{token} I-PRODUCT")  # Remaining tokens as I-PRODUCT
    
    # Process the remaining message normally
    if remaining_message:
        lines = remaining_message.split('\n')
        for line in lines:
            tokens = re.findall(r'\S+', line)  # Tokenize each line while considering non-ASCII characters
            
            for token in tokens:
                # Check if token is a price (e.g., 500 ETB, $100, or ብር)
                if re.match(r'^\d{10,}$', token):
                    labeled_tokens.append(f"{token} O")  # Label as O for "other" or outside of any entity
                elif re.match(r'^\d+(\.\d{1,2})?$', token) or 'ETB' in token or 'ዋጋ' in token or '$' in token or 'ብር' in token:
                    labeled_tokens.append(f"{token} I-PRICE")
                # Check if token could be a location (e.g., cities or general location names)
                elif any(loc in token for loc in ['Addis Ababa', 'ለቡ', 'ለቡ መዳህኒዓለም', 'መገናኛ', 'ቦሌ', 'ሜክሲኮ']):
                    labeled_tokens.append(f"{token} I-LOC")
                # Assume other tokens are part of a product name or general text
                else:
                    labeled_tokens.append(f"{token} O")
    
    return "\n".join(labeled_tokens)

# Apply the updated function to the non-null messages
df['Labeled_Message'] = df['Message'].apply(label_message_utf8_with_birr)

# Display the updated DataFrame
df.head()


In [ ]:
# Save the updated labeled dataset to a file in CoNLL format
labeled_data_birr_path = 'labeled_telegram_product_price_location.txt-'
with open(labeled_data_birr_path, 'w', encoding='utf-8') as f:
    for index, row in df.iterrows():
        f.write(f"{row['Labeled_Message']}\n\n")

In [ ]:
# categories = {
#     'kids': [
#         'toy', 'children', 'kids', 'መጫወቻ', 'play', 'games', 'fun', 'educational', 
#         'puzzle', 'doll', 'action figure', 'stuffed animal', 'arts and crafts', 
#         'books', 'outdoor toys', 'building blocks', 'baby', 'toddler', 'Baby',
#         'መጫወቻዎች'
#     ],
#     'men': [
#         'men', 'grooming', 'shaving', 'beard', 'razor', 'aftershave', 
#         'scent', 'deodorant', 'grooming kit', 'haircut', 'fashion', 'suits', 
#         'wallet', 'watch', 'accessories', 'fitness', 'shoes', 
#         'አስተካክል', 'የብርሃን ዕቃዎች'
#     ],
#     'women': [
#         'women', 'makeup', 'hair dryer', 'lipstick', 'foundation', 'mascara', 
#         'skincare', 'nails', 'jewelry', 'dresses', 'handbags', 'accessories', 
#         'fashion', 'shoes', 'perfume', 'hairstyle', 'wellness', 'beauty', 'style','Hair Drye',
#         'እንቅስቃሴ', 'የፀጉር እቃዎች', 'የውበት እቃዎች'
#     ],
#     'sport': [
#         'gym', 'GYM','fitness', 'exercise', 'እንቅስቃሴ', 'workout', 'training', 'yoga', 
#         'running', 'cycling', 'sportswear', 'equipment', 'weights', 'cardio', 
#         'aerobics', 'team sports', 'outdoor activities', 'athletics', 'health',  'workout', 'sports',
#         'ስፖርት', 'የእንቅስቃሴ መሳሪያዎች'
#     ],
#     'groceries': [
#         'food', 'snacks', 'grocery', 'ምግብ', 'produce', 'fruits', 'vegetables', 
#         'meat', 'dairy', 'bread', 'cereal', 'beverages', 'frozen', 'canned', 
#         'organic', 'bulk', 'condiments', 'spices', 'snack bars', 'breakfast', 
#         'እንቁላል', 'ወተር', 'የምግብ እቃዎች'
#     ],
#     'accessories': [
#         'jewelry', 'bags', 'accessory', 'ቀለበት', 'belts', 'hats', 'scarves', 
#         'sunglasses', 'watches', 'hair accessories', 'wallets', 'phone cases', 
#         'keychains', 'pins', 'brooches', 'fashion', 'style', 'gifts', 'decor', 'የልብስ መቶከሻ\n\n',
#         'የመልክዕ እቃዎች', 'የምታወቅ እቃዎች','Anti-theft ',' Earbuds','PowerBank','Grip Tape','humidifier'
#     ],
#     'health': [
#         'health', 'ጤና', 'wellness', 'nutrition', 'vitamins', 'supplements', 
#         'exercise', 'fitness', 'mental health', 'meditation', 'stress relief', 
#         'doctor', 'check-up', 'first aid', 'hygiene', 'immune system', 'balance', 
#         'self-care', 'አንደኛ ጤና', 'የጤና እቃዎች','pulse'
#     ],
#     'household': [
#         'cleaning', 'furniture', 'decor', 'appliances', 'utensils', 'kitchen', 
#         'bathroom', 'laundry', 'storage', 'organization', 'home improvement', 'pan', 
#         'gardening', 'tools', 'supplies', 'safety', 'maintenance', 'pets', 'spatulas','Kitchen','Mop',
#         'spatulas\n\n','nለኪችንዎ','home', 'comfort', 'ቤት', 'የቤት እቃዎች', 'እንቅስቃሴ','bottle','ፔርሙስ','knife',
#         'Glass','የላዛኛ','stove','Ironing Board','Slicer','BLENDER','MULTIFUNCTIONAL BLENDER','Toilet Brush',
#         'የቢላ ስብስብ','ቢላ','Oven','fridge', 'መጥበሻ','Toilet','Mob','cookware','Blender','KITCHENWARE','ምንጣፍ','Tablemats'
#     ]
# }


In [ ]:
def label_message_utf8_with_birr(message):
    tokens = re.findall(r'\S+', message)  # Tokenize while considering non-ASCII characters
    labeled_tokens = []
    
    for token in tokens:
        # Check if token is a price (e.g., 500 ETB, $100, or ብር)
        
        if re.match(r'^\d{10,}$', token):
            labeled_tokens.append(f"{token} O")  # Label as O for "other" or outside of any entity
        elif re.match(r'^\d+(\.\d{1,2})?$', token) or 'ETB' in token or '$' in token or 'ብር' in token:
            labeled_tokens.append(f"{token} I-PRICE")
        
        # Check if token could be a location (e.g., cities or general location names)
        elif any(loc in token for loc in ['Addis Ababa', 'ለቡ', 'ለቡ  መዳህኒዓለም', 'መገናኛ','ቦሌ','ሜክሲኮ']):
            labeled_tokens.append(f"{token} I-LOC")
        
        elif any(loc in token for loc in ['💥']):
            labeled_tokens.append(f"{token} B-Product")
        
        # Assume other tokens are part of a product name (this can be refined)
        else:
            labeled_tokens.append(f"{token} O")
    
    return "\n".join(labeled_tokens)

# Apply the updated function to the non-null messages
df['Labeled_Message'] = df['Message'].apply(label_message_utf8_with_birr)



In [ ]:
# Save the updated labeled dataset to a file in CoNLL format
labeled_data_birr_path = 'labeled_telegram_data_price_product_location_birr.txt'
with open(labeled_data_birr_path, 'w', encoding='utf-8') as f:
    for index, row in df.iterrows():
        f.write(f"{row['Labeled_Message']}\n\n")

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0")
model = AutoModelForTokenClassification.from_pretrained("masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = df['Message'][10]
ner_results = nlp(example)
print(ner_results)

In [ ]:
df['Message'][10]


In [ ]:
df

In [ ]:
df.to_csv('clean_data.csv')

In [ ]:
# Function to check if a string contains Amharic characters
def is_amharic(message):
    return bool(re.search(r'[\u1200-\u137F]', message))



In [ ]:
# Function to classify messages
def classify_message(message):
    if pd.isna(message):  # Check for NaN or None
        return 'uncategorized'
    
    if is_amharic(message):
        for category, keywords in categories.items():
            if any(keyword in message for keyword in keywords):
                return category
    else:
        for category, keywords in categories.items():
            if any(keyword in message.lower() for keyword in keywords):
                return category
    return 'uncategorized'



In [ ]:
# Apply classification to the Message column
df['Category'] = df['Message'].apply(classify_message)

# Display the updated DataFrame with categories
print(df[['Message', 'Category']])

In [ ]:
# Display counts of unique values in the Category column
category_counts = df['Category'].value_counts()
print(category_counts)


In [ ]:
df.head(100)

In [ ]:
uncategorized_items = df[df['Category'] == 'uncategorized']
uncategorized_items.head(100)

In [ ]:
df.to_csv('labeled_data.csv')

In [ ]:
uncategorized_items.to_csv('uncategorized_data.csv')